In [65]:
# Librerias generales

# Pandas
import pandas as pd
import math
pd.set_option('display.max_columns', 50) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Número máximo de filas a mostar

# Random seed
import numpy as np
np.random.seed(3301)

# Seaborn
import seaborn as sns 

# Matplolib
%matplotlib inline
import matplotlib.pyplot as plt

# math


# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

db_location = './Datos/datos_entrenamiento_laboratorio1(train_data).csv'
data = pd.read_csv(db_location, sep=',')
data.shape 

from joblib import dump, load


In [66]:
import pandas as pd
import numpy as np
from scipy.stats import skew


def eliminar_variables_correlacionadas(df, umbral=0.9):
    
    corr_matrix = df.corr().abs()
    
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    eliminadas = [column for column in upper.columns if any(upper[column] > umbral)]
    
    df_filtrado = df.drop(columns=eliminadas, errors="ignore")
    
    return df_filtrado, eliminadas

def limpiar_datos(df: pd.DataFrame, entrenamiento = False) -> pd.DataFrame:

    df = df.copy()

    # 1) market_value → precio_millones (en unidades monetarias, no “millones”)
    if 'market_value' in df.columns:
        mv = (
            df['market_value']
            .astype('string')
            .str.lower()
            .str.strip()
            .replace({'-': np.nan, 'error': np.nan})
        )
        # Captura número con posible decimal y sufijo m/k
        ext = mv.str.extract(r'€?\s*([\d]+(?:\.[\d]+)?)\s*([mk]?)', expand=True)
        ext.columns = ['cantidad', 'sufijo']
        cantidad = pd.to_numeric(ext['cantidad'], errors='coerce')  # respeta '.' como decimal
        mult = ext['sufijo'].map({'m': 1_000_000.0, 'k': 1_000.0}).fillna(1.0)
        df['precio_millones'] = cantidad * mult
    
        df = df[df['precio_millones'] < 1e10*0.1]

    # 2) Dia_partido: quitar sufijos y parsear con formato
    if 'Dia_partido' in df.columns:
        col = df['Dia_partido'].astype('string')
        for suf in [" North", " West", " Derby", " El", " Choc"]:
            col = col.str.replace(suf, "", regex=False)
        df['Dia_partido'] = pd.to_datetime(col, format="%A %B %d, %Y")

    # 3) Edad: normalizar y convertir a float
    if 'Edad' in df.columns:
        df['Edad'] = (
            df['Edad'].astype('string').str.replace('-', '.', regex=False)
        )
        df['Edad'] = df['Edad'].astype(float)  # sin errors='coerce' para replicar el lab

    # 4) Correcciones puntuales de Nacionalidad (si aplica)
    if {'Jugador', 'Nacionalidad'}.issubset(df.columns):
        df.loc[df['Jugador'].eq('Fer Lopez'), 'Nacionalidad'] = 'es ESP'
        df.loc[df['Jugador'].eq('Mateus Mane'), 'Nacionalidad'] = 'pt POR'
    
    # 5) Quitar columnas duplicadas (por contenido)
    df = df.loc[:, ~df.T.duplicated()]

    # 6) Goles entre 0 y 15
    if 'Goles' in df.columns:
        df = df[(df['Goles'] >= 0) & (df['Goles'] <= 15)]
    
    df = pd.get_dummies(df, columns=['Nacionalidad'], drop_first=False, dtype=float)

    

    # 7) eliminar correlaciones
    if entrenamiento:
        X = df.select_dtypes(include=[np.number]).drop(columns=["precio_millones"])
        y = df["precio_millones"]

        X_filtrado, eliminadas = eliminar_variables_correlacionadas(X, umbral=0.9)
        X_trans = X_filtrado.copy()
        transformadas = {}
        cols_continuas = [col for col in X_filtrado.columns if "Nacionalidad" not in col]
        correlaciones_objetivo = X_filtrado[cols_continuas].corrwith(y)
        correlaciones_ordenadas = correlaciones_objetivo.abs().sort_values(ascending=False)
        top_vars = [col for col in correlaciones_ordenadas.index if col != "Edad"][:10]

        
        for col in top_vars:
            data = X_filtrado[col]
            asimetria = skew(data)

            if asimetria < -0.5:  # cola izquierda → aplicar potencia
                X_trans[col] = np.power(data, 2)   # o np.power(data, 3)
                transformadas[col] = "potencia (x^2)"
            elif asimetria > 0.5:  # cola derecha → aplicar log1p
                X_trans[col] = np.log1p(data)
                transformadas[col] = "log1p"
            else:
                transformadas[col] = "sin cambio"

        print("Transformaciones aplicadas:")
        for k,v in transformadas.items():
            print(f"{k}: {v}")

        df_limpio = X_filtrado.copy()
        df_limpio = df_limpio.drop(columns=["%_de_regates_exitosos", "%_de_duelos_aereos_ganados"])
        df_limpio = df_limpio.copy()
        df_limpio["precio_millones"] = df["precio_millones"].values
        df_limpio = df_limpio.dropna(subset=["Edad"])

        df = df_limpio

    number_columns = df.select_dtypes(include=[np.number]).columns
    return df[number_columns]






In [67]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from typing import Optional


class FutAlpesRegressionPipeline(BaseEstimator, RegressorMixin):
    def __init__(self):
        print("Inicializando el pipeline de regresión FutAlpes...")
        self.pipeline = Pipeline([
            ('regressor', LinearRegression())
        ])

        self.model = LinearRegression()
        self.trained = False
        self.feature_columns_: Optional[list[str]] = None

    

    def fit(self, X: pd.DataFrame, y = None):
        print("Entrenando el modelo...")
        data = X.copy()
        
        dffinal = limpiar_datos(data,True)
        number_columns = dffinal.select_dtypes(include=[np.number]).columns
        data_test  = dffinal[number_columns].dropna()
        data_train = data_test.drop(columns=['precio_millones'])
        

        print(number_columns)
        
        rmse_min = math.inf
        i_test = 0

        self.feature_columns_ = data_train.columns.tolist()

        print("Buscando mejor random_state...", end='\r')
        for i in range(100):
            print(f"Probando random_state={i:03d}---", end='\r')
            X_train, X_test, y_train, y_test = train_test_split(
            data_train, data_test['precio_millones'], test_size=0.3, random_state=i
            )
            self.pipeline.fit(X_train, y_train)
            rmse = np.sqrt(mean_squared_error(y_test, self.pipeline.predict(X_test)))

            
            if rmse < rmse_min:
                rmse_min = rmse
                i_test = i
        
            

        X_train, X_test, y_train, y_test = train_test_split(
            data_train, data_test['precio_millones'], test_size=0.3, random_state=i_test
        )
        self.pipeline.fit(X_train, y_train)
        y_train_pred = self.pipeline.predict(X_train)
        y_test_pred = self.pipeline.predict(X_test)
        print()
        print("====== Model Performance ======")
        print("Train MAE:", mean_absolute_error(y_train, y_train_pred))
        print("Test MAE:", mean_absolute_error(y_test, y_test_pred))
        print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
        print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))
        print("Train R²:", r2_score(y_train, y_train_pred))
        print("Test R²:", r2_score(y_test, y_test_pred))
        print("===============================\n")

        self.pipeline.fit(data_train, data_test["precio_millones"])
        self.is_fitted_ = True
        
        return self
        
    def arreglar_datos(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        df = limpiar_datos(df)
        return df
    
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        if not self.is_fitted_:
            raise RuntimeError("El modelo no está entrenado. Llama primero a .fit().")

        df = limpiar_datos(X.copy())
        df_num = df.select_dtypes(include=[np.number]).copy()

        # Asegurar que existen todas las columnas usadas en entrenamiento
        # (si faltan, las creamos con 0; si sobran, las ignoramos)
        for col in self.feature_columns_:
            if col not in df_num.columns:
                df_num[col] = 0.0

        X_pred = df_num[self.feature_columns_].copy()

        # Cuidado: si llega a estar la columna objetivo en X, quítala
        if 'precio_millones' in X_pred.columns:
            X_pred = X_pred.drop(columns=['precio_millones'])

        # Eliminar filas con NaN para evitar errores
        X_pred = X_pred.dropna()

        return self.pipeline.predict(X_pred),X_pred

In [68]:
datos = pd.read_csv(db_location, sep=',')

modelo = FutAlpesRegressionPipeline()
modelo.fit(datos)

Inicializando el pipeline de regresión FutAlpes...
Entrenando el modelo...
Transformaciones aplicadas:
Toques_en_zona_ofensiva: log1p
Toques_en_area_rival: log1p
Acciones_que_crean_tiros: log1p
Tiros Totales: log1p
Pases_progresivos_recibidos: log1p
Acarreos_ultimo_tercio: log1p
xG: log1p
Pases_progresivos: log1p
Regates_exitosos: log1p
xA: log1p
Index(['Edad', 'Goles', 'Tiros Totales', 'xG', 'xAG',
       'Acciones_que_crean_tiros', 'Pases_intentados', 'Pases_progresivos',
       'Regates_exitosos', 'Pases_medios_completados',
       ...
       'Nacionalidad_ua UKR', 'Nacionalidad_us USA', 'Nacionalidad_uy URU',
       'Nacionalidad_uz UZB', 'Nacionalidad_ve VEN', 'Nacionalidad_wls WAL',
       'Nacionalidad_xk KVX', 'Nacionalidad_zm ZAM', 'Nacionalidad_zw ZIM',
       'precio_millones'],
      dtype='object', length=129)
Probando random_state=099---..
====== Model Performance ======
Train MAE: 12115073.80414667
Test MAE: 11883861.604463637
Train RMSE: 19037922.783544652
Test RMSE: 18

FutAlpesRegressionPipeline()

In [69]:
dump(modelo, "pipeline_final.joblib")
print("Modelo exportado como model_pipeline.joblib\n")

Modelo exportado como model_pipeline.joblib



In [70]:
pipeline = load("pipeline_final.joblib")

In [71]:
db_location = './Datos/datos_entrenamiento_laboratorio1(train_data).csv'
datos = pd.read_csv(db_location, sep=',')

pred, x_pred = pipeline.predict(datos)
x_pred["precio_millones"] = pred
x_pred["Jugador"] = datos.loc[x_pred.index, "Jugador"].values
x_pred = x_pred.loc[:, ~x_pred.columns.str.contains("Nacionalidad")]
x_pred

,Edad,Goles,Tiros Totales,xG,xAG,Acciones_que_crean_tiros,Pases_intentados,Pases_progresivos,Regates_exitosos,Pases_medios_completados,Pases_largos_completados,xA,Pases_en_ultimo_tercio,Pases_balon_muerto,Pases_al_hueco,Pases_centros,Pases_fuera_de_juego,Valla_no_vencida,Errores_defnsivos_ocasion_tiro,Toques_en_zona_ofensiva,Toques_en_area_rival,Veces_que_fue_barrido_regate,Acarreos_ultimo_tercio,Malos_controles,Perdida_balon,Pases_progresivos_recibidos,Faltas_cometidas,Duelos_aereos_ganados,precio_millones,Jugador
0,29.343,0,6,1.2,0.0,2,60,6,1,14,4,0.0,4,8,0,8,1,0,0,36,3,0,5,1,2,7,0,0,2.933531e+07,Bruno Fernandes
1,26.290,0,0,0.0,0.7,1,30,2,0,2,1,0.1,2,0,0,1,0,1,0,26,4,5,1,2,1,9,1,1,3.966251e+07,Marcus Rashford
4,20.046,0,2,0.7,0.2,3,11,0,0,3,0,0.1,0,0,0,1,0,1,0,10,3,1,0,2,1,3,0,1,3.107476e+07,Alejandro Garnacho
5,25.219,0,1,0.3,0.1,3,15,1,0,1,0,0.0,1,0,0,0,0,0,0,13,5,2,0,1,3,3,1,1,3.367085e+07,Mason Mount
6,23.086,1,1,0.1,0.0,1,9,1,0,1,0,0.0,1,0,0,0,0,0,0,4,1,0,1,1,0,1,0,0,2.443915e+07,Joshua Zirkzee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57924,22.013,0,0,0.0,0.0,1,34,0,0,12,12,0.0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.576814e+07,Bart Verbruggen
57925,21.205,1,4,0.2,0.0,2,5,0,1,2,0,0.0,0,0,1,0,0,2,0,9,2,0,2,1,3,3,4,1,3.343109e+07,Liam Delap
57931,20.306,0,2,0.1,0.0,0,32,3,2,10,5,0.0,2,2,0,1,1,1,0,21,1,0,1,4,2,4,1,1,3.704763e+07,Omari Hutchinson
57932,28.273,0,0,0.0,0.0,1,35,3,0,13,2,0.0,6,4,1,3,1,0,0,8,0,0,0,0,2,0,2,0,2.067286e+07,Kalvin Phillips


In [72]:
# Guardar solo la columna 'pred' en Resultados/prediccion.csv
x_pred[["precio_millones"]].to_csv("Resultados/prediccion.csv", index=False)

# Guardar todo el DataFrame x_pred en Resultados/tablas_completas_resultados.csv
x_pred.to_csv("Resultados/tablas_completas_resultados.csv", index=False)